In [5]:
from pathlib import Path


from matplotlib import rcParams
from matplotlib.cm import rainbow

import numpy as np
import pandas as pd


from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn import  linear_model

from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
# from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))


# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)


pd.set_option("display.max_columns", 100)




/kaggle/input/fare-classification/meta_data.csv
/kaggle/input/ml-dataset/ML data/test.csv
/kaggle/input/ml-dataset/ML data/sample_submission.csv
/kaggle/input/ml-dataset/ML data/train.csv


Definition

In [6]:
#lable encording
def encoding_label(label):
  if(label=='correct'):
    return 1
  else:
    return 0

def calculate_hours(time):
   if not(not time):
    hours=time.total_seconds()
    return hours    
        




In [7]:

train_data = pd.read_csv("/kaggle/input/ml-dataset/ML data/train.csv", index_col="tripid")
train_data["label"]=train_data["label"].apply(encoding_label).values
test_data = pd.read_csv("/kaggle/input/ml-dataset/ML data/test.csv", index_col="tripid")
submissionFormatData = pd.read_csv("/kaggle/input/ml-dataset/ML data/sample_submission.csv", index_col="tripid")
#training_data=train_data["label"]
# index_data=test_data["tripid"]
train_data.head()
# Trainng_feature=train_data[]
# trainng_feature= train_data.loc[:,[0:12]]
trainng_label= train_data[["label"]]
trainng_feature= train_data.loc[:,"additional_fare":"fare"]


# trainng_label.head()
test_data.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pickup_time,drop_time,pick_lat,pick_lon,drop_lat,drop_lon,fare
tripid,,,,,,,,,,,,
213284604,10.5,924,42,2.4486,148,2/1/2020 0:38,2/1/2020 0:53,6.83454,79.8750,6.77490,79.8840,289.27
213286352,10.5,4249,20,0.0000,91,2/1/2020 1:02,2/1/2020 2:13,6.91168,79.8723,6.55091,79.9706,1912.70
213293973,10.5,1552,255,2.6588,23,2/1/2020 5:02,2/1/2020 5:28,6.92145,79.8478,6.90539,79.8989,394.00
213294622,10.5,462,16,0.0000,198,2/1/2020 5:30,2/1/2020 5:38,6.77433,79.9416,6.80401,79.9407,154.32
213298687,10.5,814,392,12.3692,69,2/1/2020 7:00,2/1/2020 7:14,6.97968,79.9130,6.98875,79.8914,147.47


In [8]:
#feature engineering

trainng_feature['pickup_time'] = pd.to_datetime(trainng_feature['pickup_time'],format='%m/%d/%Y %H:%M')
trainng_feature['drop_time'] = pd.to_datetime(trainng_feature['drop_time'],format='%m/%d/%Y %H:%M')

trainng_feature['time_difference']=trainng_feature['drop_time']-trainng_feature['pickup_time']
trainng_feature['time_difference']=trainng_feature['time_difference'].apply(calculate_hours).values
# trainng_feature['time_difference']=trainng_feature['time_difference'].days
# trainng_feature['pyear']=trainng_feature['pickup_time'].dt.year 
# trainng_feature['pmonth']=trainng_feature['pickup_time'].dt.month 
# trainng_feature['pday']=trainng_feature['pickup_time'].dt.day
# trainng_feature['pdayofweek_num']=trainng_feature['pickup_time'].dt.dayofweek 
# trainng_feature['pHour'] = trainng_feature['pickup_time'].dt.hour 
# trainng_feature['pminute'] = trainng_feature['pickup_time'].dt.minute 

test_data['pickup_time'] = pd.to_datetime(test_data['pickup_time'],format='%m/%d/%Y %H:%M')
test_data['drop_time'] = pd.to_datetime(test_data['drop_time'],format='%m/%d/%Y %H:%M')

test_data['time_difference']=test_data['drop_time']-test_data['pickup_time']
test_data['time_difference']=test_data['time_difference'].apply(calculate_hours).values


# trainng_feature['dyear']=trainng_feature['drop_time'].dt.year 
# trainng_feature['dmonth']=trainng_feature['drop_time'].dt.month 
# trainng_feature['dday']=trainng_feature['drop_time'].dt.day
# trainng_feature['ddayofweek_num']=trainng_feature['drop_time'].dt.dayofweek 
# trainng_feature['dHour'] = trainng_feature['drop_time'].dt.hour 
# trainng_feature['dminute'] = trainng_feature['drop_time'].dt.minute 

trainng_feature=trainng_feature.drop(['pickup_time','drop_time'],axis=1)
test_data=test_data.drop(['pickup_time','drop_time'],axis=1)

trainng_feature.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,time_difference
tripid,,,,,,,,,,,
189123628,10.5,834.0,56.0,0.0000,64.0,6.86252,79.8993,6.90330,79.8783,270.32,840.0
189125358,10.5,791.0,47.0,0.0000,134.0,6.88589,79.8984,6.91373,79.8923,197.85,780.0
189125719,10.5,1087.0,80.0,0.0000,61.0,6.90839,79.8651,6.93669,79.9146,301.64,1080.0
189127273,10.5,598.0,271.0,15.6638,68.0,6.92570,79.8895,6.92748,79.8971,82.30,600.0
189128020,NaN,NaN,NaN,NaN,NaN,6.87441,79.8615,6.84478,79.9290,358.39,1020.0


Data Preprocessing 

In [9]:
numerical_cols = trainng_feature.columns[trainng_feature.dtypes != "object"].values

In [10]:
numeric_preprocessing_steps = Pipeline([
    ('standard_scaler', StandardScaler()),
    ('simple_imputer', SimpleImputer(strategy='mean'))
])

preprocessor = ColumnTransformer(
    transformers = [
        ("numeric", numeric_preprocessing_steps, numerical_cols)
    ],
    remainder = "drop"
)

In [11]:
estimators = MultiOutputClassifier(
    estimator=RandomForestClassifier(max_depth=10, random_state=0)
)

In [12]:
full_pipeline = Pipeline([
    ("preprocessor", preprocessor),
    ("estimators", estimators),
])

In [13]:
# full_pipeline

Creating the dataset

In [14]:
X_train, X_eval, y_train, y_eval = train_test_split(
    trainng_feature,
    trainng_label,
    test_size=0.3,
    shuffle=True,
    stratify= trainng_label,
    random_state=6
)
X_train.head()
# y_train.head()

,additional_fare,duration,meter_waiting,meter_waiting_fare,meter_waiting_till_pickup,pick_lat,pick_lon,drop_lat,drop_lon,fare,time_difference
tripid,,,,,,,,,,,
196858394,10.5,55.0,37.0,0.0000,410.0,6.85037,79.9221,6.84474,79.9314,60.50,60.0
191473740,10.5,1908.0,151.0,0.0000,215.0,6.99598,79.9028,6.92720,79.8443,449.33,1920.0
195369258,35.0,2579.0,492.0,28.6836,0.0,6.86733,79.8758,6.78777,79.9002,592.52,2580.0
204421493,10.5,744.0,83.0,0.0000,156.0,6.86711,79.8654,6.87280,79.8911,131.36,780.0
213456300,10.5,910.0,1.0,0.0000,228.0,6.84117,79.9654,6.87333,79.8876,357.19,900.0


Train the model

In [15]:
full_pipeline.fit(X_train, y_train)

# Predict on evaluation set
# This competition wants probabilities, not labels
preds = full_pipeline.predict(X_eval)
preds

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [16]:
print("test_probas[0].shape", preds.shape)

test_probas[0].shape (5153, 1)


In [17]:
y_preds = pd.DataFrame(
    {
        "prediction": preds[:, 0]
    },
    index = y_eval.index
)
print("y_preds.shape:", y_preds.shape)
y_preds.head()

y_preds.shape: (5153, 1)


,prediction
tripid,
204424184,1
205888602,1
202764678,1
204205375,1
205876129,1


Test model

In [18]:
f1_score(y_eval, y_preds, average=None)

array([0.55163043, 0.96551724])

In [19]:
full_pipeline.fit(trainng_feature, trainng_label)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('numeric',
                                                  Pipeline(memory=None,
                                                           steps=[('standard_scaler',
                                                                   StandardScaler(copy=True,
                                                                                  with_mean=True,
                                                                                  with_std=True)),
                                                                  ('simple_imputer',
                                                                   SimpleImputer(add_indicator=False,
                                                                 

In [20]:
test_probas = full_pipeline.predict(test_data)
test_probas

array([[1],
       [1],
       [1],
       ...,
       [1],
       [1],
       [1]])

In [21]:
# Make sure we have the rows in the same order
np.testing.assert_array_equal(test_data.index.values, 
                              submissionFormatData.index.values)


submissionFormatData["prediction"]=test_probas[:, 0]
submissionFormatData.head()

submissionFormatData.to_csv('my_submission.csv', index=True)

Show output

In [22]:
!head my_submission.csv

tripid,prediction
213284604,1
213286352,1
213293973,1
213294622,1
213298687,1
213299545,1
213302332,1
213302671,1
213305594,1
